# Projet Data Viz

In [ ]:
import altair as alt 
alt.data_transformers.disable_max_rows()
import pandas as pd
from vega_datasets import data

In [ ]:
"""
Si ce n'est pas fait, il faut charger le fichier Health_Europe_dataset_somediseases.zip 
et le countries_codes.xlsx qui sont disponibles sur le drive
"""
!unzip Health_Europe_dataset_somediseases.zip

Archive:  Health_Europe_dataset_somediseases.zip
replace hlth_cd_anr_Label.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: hlth_cd_anr_Label.csv   
  inflating: hlth_cd_anr_1_Data.csv  


In [ ]:
#Chargement et pré-traitement des données
hlth_data = pd.read_csv('hlth_cd_anr_1_Data.csv')
hlth_data['Value'] = hlth_data['Value'].apply(lambda x : x.replace(',','').replace(':','-1'))
hlth_data['Value'] = hlth_data['Value'].astype('int')
hlth_data1 = hlth_data
hlth_data1 = hlth_data1[hlth_data['AGE'] =='Total']
hlth_data1 = hlth_data1[hlth_data['SEX']=='Total']

#Calcul du nombre total de cas (sur toutes les annés) par pays et par maladie
hlth_data_grouped = hlth_data1.groupby(['GEO','ICD10'],as_index=False)['Value'].sum()

#On charge les codes de chaque pays (nécessaire pour la vizualisation de la map d'europe sur altair)
#Pas trouvé d'autre moyen pour le faire
countries_codes = pd.read_excel('countries_codes.xlsx')

#rapide prétraitement du nom des pays pour les adapters aux noms de notre dataset (des maladies)
countries_codes['Country name'] = countries_codes['Country name'].apply(lambda x : 'United Kingdom' if x =='United Kingdom of Great Britain and Northern Ireland' else x)
countries_codes['Country name'] = countries_codes['Country name'].apply(lambda x : 'France (metropolitan)' if x =='France' else x)
countries_codes['Country name'] = countries_codes['Country name'].apply(lambda x : 'Germany (until 1990 former territory of the FRG)' if x =='Germany' else x)

#On effectue un merge pour ajouter l'information des codes de chaque pays
hlth_map_data = pd.merge(hlth_data_grouped,countries_codes, left_on='GEO', right_on='Country name',how='left')
hlth_map_data_filtered = hlth_map_data.dropna()

#On effectue un pivot sur les maladies (nécessaire pour pouvoir selectioner les maladies
#dans notre visualisation)
test1 = hlth_map_data_filtered.pivot(index = ['GEO'],columns=['ICD10'],values=['Value'])
test1 = test1['Value'].reset_index()

#Remerge car l'info sur les codes pays a été perdue à cause du pivot
test2 = pd.merge(test1,countries_codes, left_on='GEO', right_on='Country name',how='left')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [ ]:
evolution_test  = hlth_data1.pivot(index = ['TIME','GEO'],columns=['ICD10'],values=['Value'])
evolution_test2 = evolution_test['Value'].reset_index()

In [ ]:
hlth_data2 = hlth_data
hlth_data2 = hlth_data2[hlth_data['SEX'] != 'Total']
hlth_data2 = hlth_data2[hlth_data['AGE'] != 'Total']

hlth_data3 = hlth_data2.groupby(['ICD10','AGE','SEX'],as_index=False)['Value'].sum()
hlth_data3 = hlth_data3.pivot(index = ['AGE','SEX'],columns=['ICD10'],values=['Value'])['Value'].reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#création de la carte

countries = alt.topo_feature(data.world_110m.url, 'countries')


columns = list(hlth_map_data_filtered['ICD10'].unique())
select_box = alt.binding_select(options=columns, name = 'column')
selection = alt.selection_single(fields=['column'], bind=select_box
                                 ,init={'column': 'Malignant neoplasms (C00-C97)'})



map = alt.Chart(countries).mark_geoshape(
    
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(test2, 'Code',list(hlth_map_data_filtered['ICD10'].unique()))
).transform_fold(
    columns, as_=['column', 'value']
).transform_filter(
    selection
).encode(
    color= 'value:Q'
).project(
    scale = 600,
    translate =[200, 900] 
).properties(
    width=700, height=500
).add_selection(selection)

evo = alt.Chart(evolution_test2).mark_line().encode(
  x='TIME:T',
  y='value:Q',
  color='GEO:N'
).transform_fold(
    columns, as_=['column', 'value']
).transform_filter(
    selection)

chart_m = alt.Chart(hlth_data3[hlth_data3['SEX']=="Males"]).mark_bar().encode(
    x=alt.X('value:Q',title='Male',sort='descending',),
    y=alt.Y('AGE:N',axis=alt.Axis(title='Age Groups'), sort='descending'),
    color=alt.Color('AGE')

).transform_fold(
    columns, as_=['column', 'value']
).transform_filter(
    selection)



chart_f = alt.Chart(hlth_data3[hlth_data3['SEX']=="Females"]).mark_bar().encode(
    x=alt.X('value:Q',sort='ascending',title='Femle'),
    y=alt.Y('AGE:N', sort='descending',axis=None),
    color=alt.Color('AGE')

).transform_fold(
    columns, as_=['column', 'value']
).transform_filter(
    selection)



In [ ]:
map & evo & (chart_m | chart_f) 

alt.VConcatChart(...)